# Implementation of new reactions to the models
In this file, we have prepared a demo of the inclusion of new reactions into the known strains of E.coli

For this first version, we have manually implemented the new reaction into the E.coli metabolic networks. This reactions appear in the dictionaries that we are going to import.

In [5]:
import cobra
from os import path
#import sys

#import dictionaries
import strainlist   # include the strains of E.coli we are using and the JSON file
import pinocembrin_path   # paths for pinocembrin production
import limonene_path     # paths for pinocembrin production

Note that the pathways have been included manually. In future versions we want to make this process completely automatic.

In [6]:
print("-"*7 ,"Reactions", "-"*7)
for r in pinocembrin_path.rxns:
    print (pinocembrin_path.rxns[r],": \t",pinocembrin_path.reactions[r])
print("-"*7 ,"Metabolites", "-"*7)
for m in pinocembrin_path.mets:
    print (pinocembrin_path.mets[m].id,":  \t", pinocembrin_path.mets[m].name)    

------- Reactions -------
EX_pc_c : 	 pc_c <=> 
MNXR7145 : 	 phe__L_c <=> nh4_c + cinnm_c 
MNXR15003 : 	 ppi_c + amp_c + cmcoa_c <=> coa_c + atp_c + cinnm_c
MNXR85703 : 	 pcc_c --> h_c + pc_c
MNXR17733 : 	 4.0 coa_c + 3.0 co2_c + pcc_c <=> 3.0 h_c + 3.0 malcoa_c + cmcoa_c
------- Metabolites -------
cmcoa_c :  	 (E)-cinnamoyl-CoA
pc_c :  	 pinocembrin
pcc_c :  	 pinocembrin chalcone


## Pinocembrin pathway
This code is divided in four parts: In the first one we include the new metabolites. Note that it is not possible to overwrite metabolites, so it is advisable to include the new metabolites before the new reactions.
In the second part we include all the new reactions, using the 'build_from_string' function
Third and fourth part are used to check the production of biomass and pinocembrin

In [7]:
#%%  PINOCEMBRIN pathway
#import the models for pinocembrin
strain_pinocembrin = {}
for s in strainlist.slist:
    strain_pinocembrin[s] = cobra.io.load_json_model(path.join('Ecoli_strains', strainlist.slist[s]))
print("models uploaded") 
#include the pinocembrin pathway new metabolites
for s in strain_pinocembrin:
    for m in pinocembrin_path.mets:
        #print(s,m)
        strain_pinocembrin[s].add_metabolites(pinocembrin_path.mets[m])      
print("Introduced new metabolites in the models")
#include the pinocembrin pathway
for s in strain_pinocembrin:
    for r in pinocembrin_path.rxns:
        #print(s,r)
        strain_pinocembrin[s].add_reaction(pinocembrin_path.rxns[r])
print("created new IDs for new reactions")
for s in strain_pinocembrin:
    for r in pinocembrin_path.rxns:
        #print(s,r)
        strain_pinocembrin[s].reactions.get_by_id(r).build_reaction_from_string(pinocembrin_path.reactions[r])      
print("Introduced new formulas for models")
# check the biomass production in all models with new reactions
print('\nBIOMASS production')
biomass_production = {}
for s in strainlist.slist:
    strain_pinocembrin[s].optimize()
    biomass_production[s] = strain_pinocembrin[s].solution.f
    print(s, ': biomass production = ', round(biomass_production[s],5))          
print('\nPINOCEMBRIN production')
pinocembrin_production = {}
for s in strainlist.slist:
    aux = strain_pinocembrin[s].objective
    strain_pinocembrin[s].change_objective('MNXR85703')
    #print(strain_pinocembrin[s].objective)
    strain_pinocembrin[s].optimize()
    pinocembrin_production[s] = strain_pinocembrin[s].solution.f
    print(s, ': pinocembrin production = ', round(pinocembrin_production[s],5))
    strain_pinocembrin[s].objective = aux
    #print(strain_pinocembrin[s].objective)

models uploaded
Introduced new metabolites in the models
created new IDs for new reactions
Introduced new formulas for models

BIOMASS production
DH1 : biomass production =  0.98248
MDS42 : biomass production =  0.7363
DH10beta : biomass production =  1.03748
MG1655 : biomass production =  0.7367
W3110 : biomass production =  0.98248
Bl21 : biomass production =  0.97561
DH5alpha : biomass production =  0.98248
MACH1 : biomass production =  0.98248

PINOCEMBRIN production
DH1 : pinocembrin production =  3.31883
MDS42 : pinocembrin production =  2.59449
DH10beta : pinocembrin production =  3.31883
MG1655 : pinocembrin production =  2.59449
W3110 : pinocembrin production =  3.31883
Bl21 : pinocembrin production =  3.31883
DH5alpha : pinocembrin production =  3.31883
MACH1 : pinocembrin production =  3.31883


We can see that with the new patways the biomass reaction and the production of pinocembrin are viable.

## Limonene pathway
This code has the same structure as the previous one, but analizes the production of limonene

In [9]:
print("-"*7 ,"Reactions", "-"*7)
for r in limonene_path.rxns:
    print (limonene_path.rxns[r],": \t",limonene_path.reactions[r])
print("-"*7 ,"Metabolites", "-"*7)
for m in limonene_path.mets:
    print (limonene_path.mets[m].id,":  \t", limonene_path.mets[m].name)    

------- Reactions -------
MNXR68753 : 	 atp_c + 5dpmev_c <=> adp_c + co2_c + ipdp_c + pi_c
MNXR2981 : 	 coa_c + mev__R_c + 2.0 nadp_c <=> 2.0 h_c + hmgcoa_c + 2.0 nadph_c
MNXR70380 : 	 grdp_c --> ppi_c + 4slim_c
MNXR341 : 	 coa_c + aacoa_c <=> 2.0 accoa_c
EX_4slim_c : 	 4slim_c <=> 
MNXR69766 : 	 atp_c + 5pmev_c <=> 5dpmev_c + adp_c
MNXR55238 : 	 dmpp_c + ipdp_c <=> grdp_c + ppi_c
MNXR69289 : 	 ipdp_c <=> dmpp_c
MNXR2982 : 	 h_c + coa_c + hmgcoa_c <=> aacoa_c + h2o_c + accoa_c
MNXR3507 : 	 atp_c + mev__R_c <=> h_c + 5pmev_c + adp_c
------- Metabolites -------
5pmev_c :  	 (R)-5-phosphomevalonate
hmgcoa_c :  	 (S)-3-hydroxy-3-methylglutaryl-CoA
4slim_c :  	 (4R)-limonene
mev__R_c :  	 (R)-mevalonate
5dpmev_c :  	 (R)-5-diphosphomevalonate


In [10]:
#%%  LIMONENE pathway
#import the models for limonene
strain_limonene = {}
for s in strainlist.slist:
    strain_limonene[s] = cobra.io.load_json_model(path.join('Ecoli_strains', strainlist.slist[s]))
print("models uploaded") 
#include the pinocembrin pathway new metabolites
for s in strain_limonene:
    for m in limonene_path.mets:
        #print(s,m)
        strain_limonene[s].add_metabolites(limonene_path.mets[m])      
print("Introduced new metabolites in the models")
#include the pinocembrin pathway
for s in strain_limonene:
    for r in limonene_path.rxns:
        #print(s,r)
        strain_limonene[s].add_reaction(limonene_path.rxns[r])
print("created new IDs for new reactions")
for s in strain_limonene:
    for r in limonene_path.rxns:
        #print(s,r)
        strain_limonene[s].reactions.get_by_id(r).build_reaction_from_string(limonene_path.reactions[r])      
print("Introduced new formulas for models")

# check the biomass production in all models with new reactions
print('\nBIOMASS production')
biomass_production = {}
for s in strain_limonene:
    strain_limonene[s].optimize()
    biomass_production[s] = strain_limonene[s].solution.f
    print(s, ': biomass production = ', round(biomass_production[s],5))
print('\nLIMONENE production')
limonene_production = {}
for s in strain_limonene:
    aux = strain_limonene[s].objective
    strain_limonene[s].change_objective('MNXR70380')
    #print(strain_pinocembrin[s].objective)
    strain_limonene[s].optimize()
    limonene_production[s] = strain_limonene[s].solution.f
    print(s, ': limonene production = ', round(limonene_production[s],5))
    strain_limonene[s].objective = aux
    #print(strain_pinocembrin[s].objective)

models uploaded
Introduced new metabolites in the models
created new IDs for new reactions
Introduced new formulas for models

BIOMASS production
DH1 : biomass production =  0.98248
MDS42 : biomass production =  0.7363
DH10beta : biomass production =  1.03748
MG1655 : biomass production =  0.7367
DH5alpha : biomass production =  0.98248
Bl21 : biomass production =  0.9779
W3110 : biomass production =  0.98248
MACH1 : biomass production =  0.98248

LIMONENE production
DH1 : limonene production =  4.12702
MDS42 : limonene production =  3.20115
DH10beta : limonene production =  4.12702
MG1655 : limonene production =  3.20115
DH5alpha : limonene production =  4.12702
Bl21 : limonene production =  4.10538
W3110 : limonene production =  4.12702
MACH1 : limonene production =  4.12702


We can see that with the new patways the biomass reaction and the production of limonene are viable.